In [17]:
import pandas as pd
import numpy as np
import pickle
from scipy import interpolate
from scipy.optimize import brentq

In [18]:
a = !ls /scratch/jdg1g14/all_resultspc1/vox*
b = !ls /scratch/jdg1g14/all_resultspc2/vox*
files = a+b
files

['/scratch/jdg1g14/all_resultspc1/vox_1.5_1_D0_S0.mat',
 '/scratch/jdg1g14/all_resultspc1/vox_1.5_1_D0_S1.mat',
 '/scratch/jdg1g14/all_resultspc1/vox_1.5_1_D1_S0.mat',
 '/scratch/jdg1g14/all_resultspc1/vox_1.5_1_D1_S1.mat',
 '/scratch/jdg1g14/all_resultspc1/vox_1.5_2_D0_S0.mat',
 '/scratch/jdg1g14/all_resultspc1/vox_1.5_2_D0_S1.mat',
 '/scratch/jdg1g14/all_resultspc1/vox_1.5_2_D1_S0.mat',
 '/scratch/jdg1g14/all_resultspc1/vox_1.5_2_D1_S1.mat',
 '/scratch/jdg1g14/all_resultspc1/vox_1.5_3_D0_S0.mat',
 '/scratch/jdg1g14/all_resultspc1/vox_1.5_3_D0_S1.mat',
 '/scratch/jdg1g14/all_resultspc1/vox_1.5_3_D1_S0.mat',
 '/scratch/jdg1g14/all_resultspc1/vox_1.5_3_D1_S1.mat',
 '/scratch/jdg1g14/all_resultspc1/vox_1.5_4_D0_S0.mat',
 '/scratch/jdg1g14/all_resultspc1/vox_1.5_4_D0_S1.mat',
 '/scratch/jdg1g14/all_resultspc1/vox_1.5_4_D1_S0.mat',
 '/scratch/jdg1g14/all_resultspc1/vox_1.5_4_D1_S1.mat',
 '/scratch/jdg1g14/all_resultspc1/vox_1.5_5_D0_S0.mat',
 '/scratch/jdg1g14/all_resultspc1/vox_1.5_5_D0_S

In [19]:
sig_coeffs = pickle.load( open( "ERF_Sigmoid_Coeffs.p", "rb" ) )

In [20]:
def PSF(coeffs):
    
    rss = np.linspace(25,35,100)
    
    Y = sigmoid(rss,*coeffs)
    PSF = np.diff(Y)
    
    f = interpolate.interp1d(np.linspace(25,35,99),-PSF/max(-PSF))
    
    def opti_f(x):
        return f(x) - 0.5
    
    half_width = brentq(opti_f,30,35)
    
    return half_width,PSF

def sigmoid(x, a,x0, k,d):
    y = a / (1 + np.exp(-k*(x-x0))) + d
    return y

In [21]:
fout = open('halfwidths_PSF.dat','w')
PSFs = []
for ind,file in enumerate(files):
    fout.write(file)
    PSF_file = []
    for i in range(3):
        width,modtransfunc = PSF(sig_coeffs[ind][i])
        fout.write(','+str(width))
        PSF_file.append(modtransfunc)
    fout.write('\n')
    PSFs.append(PSF_file)
fout.close()

In [22]:
df2 = pd.read_csv('halfwidths_PSF.dat',header=None,names=['name','xslice','yslice','zslice','Index'])
df2.Index = df2.index

df2['mag'] = df2.name.apply(lambda x : float(x.split('/')[-1].split('_')[1]))
df2['exp'] = df2.name.apply(lambda x : float(x.split('/')[-1].split('_')[2]))
df2['Soffset'] = df2.name.apply(lambda x : 1.0 if x.split('/')[-1].split('_')[4][1] == '1' else 0.0)
df2['Doffset'] = df2.name.apply(lambda x : 1.0 if x.split('/')[-1].split('_')[3][1] == '1' else 0.0)
df2.xslice = df2.xslice.apply(lambda x : np.abs(30-x))
df2.yslice = df2.yslice.apply(lambda x : np.abs(30-x))
df2.zslice = df2.zslice.apply(lambda x : np.abs(30-x))
df2.head()

,name,xslice,yslice,zslice,Index,mag,exp,Soffset,Doffset
0,/scratch/jdg1g14/all_resultspc1/vox_1.5_1_D0_S...,0.344026,0.467851,0.467848,0,1.5,1,0,0
1,/scratch/jdg1g14/all_resultspc1/vox_1.5_1_D0_S...,0.527367,0.576481,0.576794,1,1.5,1,1,0
2,/scratch/jdg1g14/all_resultspc1/vox_1.5_1_D1_S...,0.826644,0.827007,0.830890,2,1.5,1,0,1
3,/scratch/jdg1g14/all_resultspc1/vox_1.5_1_D1_S...,0.896168,0.897161,0.896359,3,1.5,1,1,1
4,/scratch/jdg1g14/all_resultspc1/vox_1.5_2_D0_S...,0.344029,0.467850,0.467851,4,1.5,2,0,0


In [23]:
df2.to_pickle('PSFHalfPolar.p')